<a href="https://colab.research.google.com/github/georgeaidinis/NTUA/blob/master/NN/Lab%201/Lab_1_Small_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Αϊδίνης Γιώργος 03116031

Κολιός Παναγιώτης 03116100

---

Ομάδα M.B.8

---


#Lab 1: Επιβλεπόμενη Μάθηση - Ταξινόμηση - Μικρό Dataset (S11 - Quality Assessment of Digital Colposcopies)

Αρχικά ενημερώνουμε τις βιβλιοθήκες που θα χρησιμοποιήσουμε.



In [0]:
!pip install --upgrade pip #upgrade pip package installer
!pip install scikit-learn --upgrade #upgrade scikit-learn package
!pip install numpy --upgrade #upgrade numpy package
!pip install pandas --upgrade #--upgrade #upgrade pandas package



1.   Το dataset περιγράφει μετρήσεις κολποσκοπήσεων και την κατάσταση (ετικέτα) των αντίστοιχων κόλπων, όπως προκύπτει από τις εκτιμήσεις καθενός από 6 ειδικούς και την εκτίμηση της πλειοψηφίας. Οι μετρήσεις και οι εκτιμήσεις είναι διαφορετικές για τις διαφορετικές μεθόδους με τις οποίες πραγματοποιούνται οι κολποσκοπήσεις: Hinselmann, Green, Schiller.



2.    Έχουμε συνολικά 287 δείγματα, 92 από τη μέθοδο Schiller, 98 από τη μέθοδο Green και 97 από τη μέθοδο Hinselmann. Σε κάθε δείγμα περιέχονται μετρήσεις για 62 χαρακτηριστικά-παρατηρήσεις από τα οποία προέκυψαν οι εκτιμήσεις. Όλα τα χαρακτηριστικά είναι διατεταγμένα.Οι μετρήσεις είναι αριθμητικά δεδομένα και αφορούν τιμές όπως τις επιφάνειες περιοχών του κόπλου.

3.    Υπάρχουν επικεφαλίδες στην πρώτη γραμμή πάνω από τα χαρακτηριστικά και τις ετικέτες, οι οποίες θα πρέπει να αφαιρεθούν. Δεν υπάρχει στήλη για την αρίθμηση των γραμμών.

4.    Η τιμή της κατάστασης μπορεί να πάρει δύο τιμές, 0 για κακή και 1 για καλή. Όπως υποδεικνύεται από τις FAQ το πρόβλημα θα αναλυθεί ως binary classification λαμβάνοντας υπόψην μόνο τις εκτιμήσεις της πλειοψηφίας. Έτσι οι στήλες των ετικετών που αφορούν μεμονωμένα τον κάθε ειδικό αφαιρούνται και μένει μόνο η τελευταία στήλη με τις εκτιμήσεις της πλειοψηφίας. Η στήλη αυτή είναι η τελευταία (θέση 69 στον αρχικό πίνακα). Παρατηρούμε οτι όλες οι ισοψηφίες (3-3) επιλύονται θεωρώντας την κατάσταση ως καλή, γεγονός που ίσως μας αναγκάσει να αυξήσουμε αργότερα το πλήθος των δειγμάτων με ετικέτα 0 ή να διαγράψουμε δείγματα με ετικέτα 1.

    
5.    Συνενώνουμε τα αρχεία που αφορούν τις 3 διαφορετικές μεθόδους. Μετά τη συνένωση οι επικεφαλίδες προστέθηκαν ως γραμμές, με αποτέλεσμα να έχουμε 3 φορές την ίδια γραμμή με τις επικεφαλίδες, τις οποίες και αφαιρούμε.

6.    Δεν υπάρχουν απουσιάζουσες τιμές.



In [0]:
from urllib.request import urlretrieve
import os
import zipfile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")


In [3]:
download("http://archive.ics.uci.edu/ml/machine-learning-databases/00384/Quality%20Assessment%20-%20Digital%20Colposcopy.zip","QADC.zip")
print("All the files are downloaded")

Download file... QADC.zip ...
File downloaded
All the files are downloaded


In [4]:
!ls

QADC.zip  sample_data


In [0]:
!unzip -q "QADC.zip"

In [6]:
!ls

 QADC.zip  'Quality Assessment - Digital Colposcopy'   sample_data


Rename directory for better handling



In [0]:
!mv Quality\ Assessment\ -\ Digital\ Colposcopy QADC

Remove attributes line for the 2nd and 3rd files:

In [0]:
with open("QADC/hinselmann.csv",'r') as f:
    with open("QADC/hinselmann1.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

In [0]:
with open("QADC/schiller.csv",'r') as f:
    with open("QADC/schiller1.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

create one big .csv

In [0]:
!cat QADC/green.csv QADC/hinselmann1.csv QADC/schiller1.csv > all.csv

We needed to concatenate the 3 .csv files && remove the category names from the second and third files gia na min exoume asxeta instances
We also need to remove the attribute of the experts, apart from the concensus

In [0]:
import pandas as pd
import numpy as np

df = pd.read_csv("all.csv")
df = df.drop(df.columns[[62, 63, 64, 65, 66, 67]], axis=1)

Let's print number, names and types of attributes

In [23]:
print ("We have ",len(df.columns), " attributes.")
for i in range(0, len(df.columns)):
    print('{:<10}{:<40}{:<10}{:<20}'.format(str(i+1), str(df.columns[i]),"type: ", str(df.dtypes[df.columns[i]])))


We have  63  attributes.
1         cervix_area                             type:     float64             
2         os_area                                 type:     float64             
3         walls_area                              type:     float64             
4         speculum_area                           type:     float64             
5         artifacts_area                          type:     float64             
6         cervix_artifacts_area                   type:     float64             
7         os_artifacts_area                       type:     float64             
8         walls_artifacts_area                    type:     float64             
9         speculum_artifacts_area                 type:     float64             
10        cervix_specularities_area               type:     float64             
11        os_specularities_area                   type:     float64             
12        walls_specularities_area                type:     float64             
13 

We need to count the occurences of each label (healthy or unhealthy):

In [13]:
labels_df = df.iloc[:, [62]]
labels = labels_df.values.reshape(287,)
positive = 0
negative = 0
for i in range(0, len(labels)):
    positive += labels[i]==1
    negative += labels[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  216  negative:  71


Βλεπουμε οτι εχουμε unbalanced data set, καθως η κλαση των positive(υγειων) ειναι κατα πολυ μεγαλυτερη σε μεγεθος απο την κλαση των negative. (75-25)

Train - Test split

In [125]:
from sklearn.model_selection import train_test_split


train, test, train_labels, test_labels = train_test_split(df, labels, test_size=0.2)
print(len(train))


229


In [126]:
from scipy import stats as st


# standardization
std_x = st.zscore(train)
Xvar = np.var(std_x, axis=0)
print(Xvar)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Undersampling - Oversampling

In [127]:
positive = 0
negative = 0
for i in range(0, len(train_labels)):
    positive += train_labels[i]==1
    negative += train_labels[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  169  negative:  60


In [128]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(sampling_strategy=0.65)
train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [129]:
positive = 0
negative = 0
for i in range(0, len(train_labels_oversampled)):
    positive += train_labels_oversampled[i]==1
    negative += train_labels_oversampled[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  169  negative:  109


In [130]:
from imblearn.under_sampling import RandomUnderSampler

ros = RandomUnderSampler(random_state=0)
train_undersampled, train_labels_undersampled = ros.fit_sample(train_oversampled,train_labels_oversampled)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [131]:
positive = 0
negative = 0
for i in range(0, len(train_labels_undersampled)):
    positive += train_labels_undersampled[i]==1
    negative += train_labels_undersampled[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  109  negative:  109


In [132]:
Xvar = np.var(train_undersampled, axis=0)
print(Xvar)

[6.30904381e-02 3.71779851e-05 5.17588274e-02 3.45886080e-02
 4.92078009e-03 4.12846440e-03 1.85240636e-02 1.59511273e-02
 7.35949723e-03 6.72240073e-04 1.67196037e-03 4.70183725e-04
 1.11578612e-02 1.36595893e-03 1.89294819e-02 2.42966946e+03
 3.03317566e+02 1.53148104e+03 3.93449300e+03 3.30142694e+03
 2.84963831e+02 2.24281866e+03 4.92996287e+03 4.05927984e+03
 5.19511925e+02 2.30130760e+03 6.85627592e+03 1.17879914e+03
 4.49947951e+02 5.51363551e+02 2.70613063e+03 1.29852116e+03
 4.93612301e+02 6.32529002e+02 2.95173793e+03 2.47173961e+03
 5.87125163e+02 1.12041810e+03 4.99731145e+03 5.49135908e-01
 4.45608194e-01 3.43832296e+03 4.68797860e+02 4.24602627e+03
 4.45364911e+02 4.48090584e-01 6.36383459e-02 3.01604617e+03
 4.13602552e+02 1.72150097e+03 4.55085570e+02 4.53760412e-02
 6.16136892e-02 4.17285730e-02 7.02754240e-02 2.46292278e-02
 2.09082826e-01 4.80350378e-02 5.62237445e-02 1.03195608e+04
 6.96535626e-02 3.45979614e-02 2.50000000e-01]


In [138]:
from scipy import stats as st
from sklearn import preprocessing


train_scaled = preprocessing.scale(train_undersampled)

Xvar = np.var(train_scaled, axis=0)
print(Xvar)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [137]:
Yvar = np.var(np.asarray(test), axis=0)
print(Yvar)

[4.03496918e-02 4.63873100e-05 3.94788130e-02 3.02162360e-02
 3.29430285e-03 5.38331944e-03 2.88393877e-02 7.82565493e-03
 1.50640780e-02 1.28816752e-03 5.60647379e-03 3.78570437e-03
 1.88683092e-02 1.50749170e-03 2.03016954e-02 2.98263096e+03
 2.82069045e+02 2.07708767e+03 4.45231234e+03 2.14381566e+03
 2.41248262e+02 1.56059768e+03 3.20953016e+03 4.56381198e+03
 4.47692376e+02 3.06611469e+03 6.95689401e+03 1.41129377e+03
 4.35261628e+02 7.20855883e+02 2.97225492e+03 1.02516706e+03
 3.19590765e+02 6.43645132e+02 2.04587052e+03 2.37947717e+03
 5.30279612e+02 1.23116451e+03 4.58834906e+03 4.77859861e-01
 3.16317404e-01 3.29231847e+03 4.64338146e+02 3.33091773e+03
 3.41033475e+02 3.97294543e-01 6.24156947e-02 3.46769744e+03
 3.85283186e+02 1.29817445e+03 2.82277554e+02 3.14114104e-02
 3.98653690e-02 2.70736240e-02 4.45784640e-02 1.62156646e-02
 1.21072691e-01 3.32473717e-02 3.69132900e-02 4.84253370e+03
 6.94157766e-02 3.11866770e-02 1.53686088e-01]


In [139]:
scaler = preprocessing.StandardScaler().fit(train_undersampled)
test_scaled = scaler.transform(test)
Yvar = np.var(np.asarray(test_scaled), axis=0)
print(Yvar)

[0.6395532  1.24770909 0.76274551 0.87358925 0.6694676  1.30395201
 1.55686076 0.490602   2.04688956 1.91623138 3.35323367 8.05154277
 1.69103279 1.10361422 1.07249081 1.22758713 0.92994629 1.35626078
 1.13161018 0.64936032 0.84659257 0.69581982 0.65102522 1.12429105
 0.86175573 1.33233588 1.01467533 1.19723007 0.96735995 1.30740576
 1.09834126 0.78948814 0.647453   1.0175741  0.6931071  0.96267308
 0.90317984 1.09884382 0.91816352 0.87020327 0.70985545 0.95753613
 0.99048691 0.78447883 0.76573944 0.8866389  0.98078751 1.14974945
 0.93153    0.75409452 0.62027358 0.6922466  0.6470213  0.64880302
 0.63433931 0.65839111 0.57906569 0.69214834 0.65654272 0.46925773
 0.99658616 0.90140216 0.61474435]
